In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('%s/../' % os.getcwd())

import datetime
import pandas as pd
import numpy as np
import getpass

In [2]:
import formulas_auxiliares_ajuste_base as aux
from formulas_auxiliares_ticker import *

In [3]:
pasta = r'C:\Users\correigu\OneDrive - Banco BTG Pactual S.A\Desktop\aqruivos_offshore'

In [4]:
data_inicio = pd.to_datetime('2018-07-01')
data_final = pd.to_datetime('2018-07-15')

In [5]:
pb, calypso, calypso_1 = aux.TratamentoDosDados(pasta, data_inicio, data_final).tabelas_ajustadas()
calypso = aux.ajuste_contas_calypso(calypso)
ticker_diffs = pd.read_excel(pasta+'\\'+ 'tickers_diffs.xlsx')

Povoando lista de arquivos que serão baixados para a memória
    O arquivo Ajustes não possui data registrada
    O arquivo Ajustes não pôde ser comparado com os demais
    O arquivo tickers_diffs.xlsx não possui data registrada
    O arquivo tickers_diffs.xlsx não pôde ser comparado com os demais
--------------------------------------------------------------------------------------------------------------
Baixando arquivos!!
    10 arquivos do ubs importados
    20 arquivos da morgan importados
    10 arquivos do Calypso PROP+ASSET importados
    10 arquivos do Calypso BD+WM importados
    10 arquivos do Calypso INTC importados
--------------------------------------------------------------------------------------------------------------


In [6]:
mapa_tickers = pb[['ticker','isin','cusip','sedol']].drop_duplicates().reset_index(drop=True).fillna('')

In [7]:
lista_tickers = [get_ticker_bases(calypso, mapa_tickers, x) for x in mapa_tickers.index]

O ticker FOUREC não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
Nós aconselhamos que você confira o ticker FOUREC, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você deverá fazer o ajuste na próxima parte.
O ticker GOLD não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
O ticker GOLD não foi encontrado pelo nome.
 Ele será então contruído pelo Ticker + Isin
O ticker CRFFFA não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
Nós aconselhamos que você confira o ticker CRFFFA, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você deverá fazer o ajuste na próxima parte.


In [8]:
mapeados = pd.DataFrame(lista_tickers).rename(columns={0:'ticker_calypso', 1:'construct'})

In [9]:
mapa_tickers = pd.concat([mapa_tickers, mapeados], axis=1)

In [10]:
ticker_diffs = ticker_diffs.rename(columns={
    'Stock':'ticker',
    'Isin':'isin',
    'Cusip':'cusip',
    'Sedol':'sedol'
})

In [11]:
mapa_tickers_diffs = ticker_diffs[['ticker','isin','cusip','sedol']].drop_duplicates().reset_index(drop=True)

In [12]:
lista_tickers_diffs = [get_ticker_bases(calypso, mapa_tickers_diffs, x) for x in mapa_tickers_diffs.index]

O ticker ABEV não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
Nós aconselhamos que você confira o ticker ABEV, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você deverá fazer o ajuste na próxima parte.
O ticker BBDO não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
Nós aconselhamos que você confira o ticker BBDO, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você deverá fazer o ajuste na próxima parte.
O ticker BBVA não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
Nós aconselhamos que você confira o ticker BBVA, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você deverá fazer o ajuste na próxima parte.
O ticker BUD não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
Nós aconselhamos que você confira o ticker BUD, porque ele foi ajustado pelo Isin.
Caso ele esteja ERRADO você deverá fazer o ajuste na próxima parte.
O ticker CS não foi encontrado pelos códigos.
 Tentaremos agora pelo nome
O ticker CS não 

In [13]:
mapeados_diff = pd.DataFrame(lista_tickers_diffs).rename(columns={0:'ticker_calypso', 1:'construct'})

In [14]:
mapa_tickers_diffs = pd.concat([mapa_tickers_diffs, mapeados_diff], axis=1)

In [15]:
mapa_tickers = mapa_tickers.drop(['isin', 'cusip', 'sedol', 'construct'], axis=1)

In [16]:
tickers_diffs = mapa_tickers_diffs[['ticker_calypso','isin','cusip']].rename(columns={'ticker_calypso':'ticker'}).copy()

In [17]:
pb['id'] = range(0, len(pb))

In [18]:
pb_1 = pd.merge(pb, mapa_tickers, on='ticker', how='left').drop_duplicates()

In [19]:
# Isso aqui tem que ser zero para estar tudo ok!
check_1 = pb_1[~pb_1['id'].isin(pb['id'])].shape[0]+pb[~pb['id'].isin(pb_1['id'])].shape[0]
if check_1 == 0:
    pb = pb_1.drop(['ticker', 'isin','cusip','sedol','id'], axis=1)[['conta', 'pb', 'data_ref','ticker_calypso', 'currency',
                                                                    'qt', 'accrual', 'month','year']].rename(columns={
        'ticker_calypso':'ticker'
    })
else:
    print('Houve algum problema na hora do mapeamento! será necessário fazer ajustes')

In [20]:
mes = pb.month.drop_duplicates()[0]
ano = pb.year.drop_duplicates()[0]
folder = str(mes)+str(ano)+ r'\consolidado'

subpasta = '\\\\DRIOC0231PFS\\Apoio_SB\\Geral\\BD\\Aluguel_Offshore\\Arquivos diários\\'
subpasta+folder
if os.path.isdir(subpasta+folder)==False:
    os.mkdir(subpasta+folder)

pasta_final = subpasta+folder

In [21]:
pasta_final

'\\\\DRIOC0231PFS\\Apoio_SB\\Geral\\BD\\Aluguel_Offshore\\Arquivos diários\\jul2018\\consolidado'

In [22]:
writer = pd.ExcelWriter(pasta_final+'\\bases_batimento_pb_calypso.xlsx')

In [23]:
pb.to_excel(writer, sheet_name='base_pb', index=False)
calypso.to_excel(writer, sheet_name='base_calypso', index=False)
tickers_diffs.to_excel(writer, sheet_name='ticker_diffs', index=False)
writer.save()